<a href="https://colab.research.google.com/github/Rozi737/Wheather-Checker/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_groq langchain_core langchain_community

In [2]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
result = llm.invoke("Who is lord ganesha?")
print(result.content)

Lord Ganesha, also known as Ganapati, is one of the most revered and beloved deities in Hinduism. He is the remover of obstacles, the patron of knowledge and wisdom, and the god of good fortune. Ganesha is often depicted as a rotund figure with an elephant's head, a large belly, and four arms.

**Mythological Background:**
Ganesha is the son of Lord Shiva, the destroyer of the universe, and Goddess Parvati, the embodiment of feminine energy. According to myth, Parvati created Ganesha from clay and breathed life into him. When Shiva returned home, he did not recognize Ganesha and beheaded him in a fit of rage. Parvati was grief-stricken, and Shiva, realizing his mistake, sent his attendants to find the head of the first living being they encountered, which happened to be an elephant. The elephant's head was attached to Ganesha's body, and he was restored to life.

**Attributes and Symbolism:**
Ganesha is often depicted with the following attributes:

1. **Elephant Head:** Represents wis

In [3]:
!pip install pypdf

In [4]:
!pip install chromadb

In [5]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [6]:
create_vector_db()

NameError: name 'create_vector_db' is not defined

In [ ]:
import textwrap  # To wrap long text lines for better visibility

# Required imports
from huggingface_hub import login
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os


# Initialize LLM
def initialize_llm():
    llm = ChatGroq(
        temperature=0,
        groq_api_key="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",
        model_name="llama-3.3-70b-versatile"
    )
    return llm

# Create Chroma Vector DB
def create_vector_db():
    loader = DirectoryLoader("/content/data/", glob='*.pdf', loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
    vector_db.persist()

    print("ChromaDB created and data saved")

    return vector_db

# Setup QA Chain
def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
    PROMPT = PromptTemplate(template=prompt_templates, input_variables=['context', 'question'])

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

# Function to print wrapped text
def print_wrapped(text, width=80):
    wrapped_text = textwrap.fill(text, width=width)
    print(wrapped_text)

# Main Loop
def main():
    print("Initializing Chatbot.........")
    llm = initialize_llm()

    db_path = "/content/chroma_db"

    if not os.path.exists(db_path):
        vector_db = create_vector_db()
    else:
        embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
        vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

    qa_chain = setup_qa_chain(vector_db, llm)

    while True:
        print("\n" + "-" * 50)  # Separator for user clarity
        query = input("\nHuman: ")
        if query.lower() == "exit":
            print("\nChatbot: Take Care of yourself, Goodbye!")
            print("-" * 50)
            break
        print("\nProcessing your query...")  # Show query is being processed
        response = qa_chain.run(query)
        print("\n" + "-" * 50)  # Separator for chatbot response
        print_wrapped(f"Chatbot: {response}")  # Use wrapped text for better readability
        print("-" * 50)  # End separator

if __name__ == "__main__":
    main()

Initializing Chatbot.........


<ipython-input-8-500eb2c2c552>:29: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-8-500eb2c2c552>:31: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


ChromaDB created and data saved

--------------------------------------------------

Processing your query...


<ipython-input-8-500eb2c2c552>:82: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)



--------------------------------------------------
Chatbot: Hello. It seems like you've shared a passage about a research study,
and then there's a quote from a participant named Anna, who's expressing her
appreciation for the support she received from an AI system. It's beautiful to
see how Anna felt heard and helped by the AI, and how she values the acceptance
and support it provided.   As a compassionate mental health chatbot, I'm here to
listen and offer support. If you'd like to talk about what's on your mind, I'm
here to provide a non-judgmental and empathetic space. What's been going on, and
how can I support you today?
--------------------------------------------------

--------------------------------------------------

Processing your query...

--------------------------------------------------
Chatbot: I'm so sorry to hear that you're feeling sad. It takes a lot of courage
to acknowledge and share your emotions, and I'm here to listen and support you
without judgment. It so

In [ ]:
from google.colab import drive
drive.mount('/content/drive')